<a href="https://colab.research.google.com/github/abhisheksingh1234/DataScience/blob/master/LSTM_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive to '/content/drive' directory
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Path to the file in Google Drive after mounting
import shutil

file_path = '/content/drive/My Drive/winemag-data_first150k.csv'
with open('data2.txt','w+') as file_w:
  with open(file_path, 'r') as file:
      line_count = 0
      for line in file:
          parts = line.split(',')
          description = parts[2]
          description = description.strip('"')
          file_w.write(description)
          file_w.write("\n")
          print(description)

          line_count +=1
          if line_count >= 1000:
            break

shutil.copy('data2.txt','/content/drive/My Drive/data3.txt')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import tensorflow as tf
import re
from tensorflow.keras import layers

def pad_punctuation(text):
    # Using regular expression to split text based on punctuation while keeping punctuation as separate words
    padded_text = re.sub(r"([{}])".format(re.escape(string.punctuation)), r' \1 ', text)
    return padded_text

data_list = []

with open('/content/drive/My Drive/data3.txt', 'r') as file:
      for line in file:
      ## Pad Puntation to treat them as a sperate words
        print(line)
        data_list = file.readlines()

text_data = [pad_punctuation(x) for x in data_list]

## Convert it into a tensor flow dataset

text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

## Keras Text Vectorization Layer, convert to lower case, most prevalent
## 10000 words trim or pad token to 201 token long.

vectorized_layer = layers.TextVectorization(
    standardize = 'lower',
    max_tokens = 10000,
    output_mode = "int",
    output_sequence_length = 200+1
)

## Apply Text Vectorized layer to the training data

vectorized_layer.adapt(text_ds)

## Vocab variables stores a list of the word tokens
vocab = vectorized_layer.get_vocabulary()


In [ ]:
import tensorflow as tf

def prepare_inputs(text):
  text = tf.expand_dims(text, -1)
  tokenized_sentences = vectorized_layer(text)
  x = tokenized_sentences[:, :-1]
  y = tokenized_sentences[:, 1:]
  return x,y

train_ds = text_ds.map(prepare_inputs)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

inputs = layers.Input(shape=(None,), dtype = "int32")
## Embedding Layer Size of Vocublary and dimensionality of the embedding vector
x = layers.Embedding(10000,100)(inputs)

x = layers.LSTM(128,return_sequences=True)(x)

##Dense Layer transforms the hidden states at each timestamp into vector of probabilities for each token
outputs = layers.Dense(10000, activation = 'softmax')(x)
lstm = Model(inputs,outputs)

loss_fn = SparseCategoricalCrossentropy()
lstm.compile("adam",loss_fn)


In [42]:
import matplotlib.pyplot as plt

# Train the model and store the training history
history = lstm.fit(train_ds,epochs=25)
# Save the trained model
model_path = '/content/drive/My Drive/'
lstm.save(model_path)

Epoch 1/25
32/32 [==============================] - 76s 2s/step - loss: 0.1543
Epoch 2/25
32/32 [==============================] - 76s 2s/step - loss: 0.1512
Epoch 3/25
32/32 [==============================] - 75s 2s/step - loss: 0.1481
Epoch 4/25
32/32 [==============================] - 75s 2s/step - loss: 0.1458
Epoch 5/25
32/32 [==============================] - 75s 2s/step - loss: 0.1439
Epoch 6/25
32/32 [==============================] - 76s 2s/step - loss: 0.1418
Epoch 7/25
32/32 [==============================] - 75s 2s/step - loss: 0.1388
Epoch 8/25
32/32 [==============================] - 76s 2s/step - loss: 0.1366
Epoch 9/25
32/32 [==============================] - 75s 2s/step - loss: 0.1341
Epoch 10/25
32/32 [==============================] - 76s 2s/step - loss: 0.1328
Epoch 11/25
32/32 [==============================] - 75s 2s/step - loss: 0.1308
Epoch 12/25
32/32 [==============================] - 77s 2s/step - loss: 0.1281
Epoch 13/25
32/32 [==============================

In [49]:
import numpy as np
from tensorflow.keras import callbacks

class TextGenerator(callbacks.Callback):
    def __init__(self, model, index_to_word, top_k=10):
        super().__init__()
        self.model = model
        self.index_to_word = index_to_word
        self.word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.model.predict(x)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append({'prompt': start_prompt, 'word_probs': probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + ' ' + self.index_to_word[sample_token]
        print(f"\n generated text: \n {start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/')
text_gen_callback = TextGenerator(loaded_model, index_to_word)
text_gen_callback.on_epoch_end(epoch=0)